In [61]:
"""
01_check_and_enrich.ipynb
This notebook checks the data and enriches it with additional information.
"""

'\n01_check_and_enrich.ipynb\nThis notebook checks the data and enriches it with additional information.\n'

In [62]:
### IMPORTS ###
# WARNING: install xlrd module (pip install xlrd) to read XLS file (Excel 2003), install openpyxl module (pip install openpyxl) to read XLSX file (Excel 2007+)
import pandas as pd
from pathlib import Path
from datetime import datetime
from functions import create_output_directory, load_json_to_dict
import numpy as np
import networkx as nx
from math import log2
from collections import Counter
import matplotlib.pyplot as plt

In [63]:
### GLOBALS ###
### Input constants ###
DATA_IN_DIR = "data_in"
# CSV
FILE_CSV_IN = "EVENT-LOG_ED_ENG.csv"
FILE_CSV_IN_DTYPE = "EVENT-LOG_ED_ENG.json"                         # dtype file
CSV_SEP = ";"
LOG_KEY = "CaseID"
LOG_OUTCOME = "OUTCOME"
LOG_ACTIVITY = "ACTIVITY"
LOG_TIMESTAMP = "TIMESTAMP"

# XLS
FILE_XLS_IN = "Estrazione_Step_PS_ConSchedaCriptato.xls"
FILE_XLS_IN_DTYPE = "Estrazione_Step_PS_ConSchedaCriptato.json"     # dtype file
XLS_KEY = "Chiave"

### Output constants ###
DATA_OUT_DIR = "data_out"

In [64]:
shifts_dict = {
    "Morning": (7, 15),       # 07:00 - 14:59
    "Evening": (15, 22),  # 15:00 - 21:59
    "Night": (22, 7)      # 22:00 - 06:59
}

## FUNCTIONS

In [65]:
def count_rows_per_case(dataframe: pd.DataFrame, column_name: str, column_name_out: str) -> pd.DataFrame:
    """
    Returns a dataframe with the count of rows for each distinct case_id.

    Parameters:
        dataframe (pd.DataFrame): The input dataframe.
        column_name (str): The name of the column containing case IDs.
        column_name_out (str): The name of the output column for case IDs.
    Returns:
        pd.DataFrame: A dataframe with two columns: column_name and 'case_row_count'.
    """
    if column_name not in dataframe.columns:
        raise ValueError(f"The column '{column_name}' does not exist in the dataframe.")
    
    result = dataframe.groupby(column_name).size().reset_index(name='case_row_count')
    result.rename(columns={column_name: column_name_out}, inplace=True)
    return result

In [66]:
def enrich_concurrent_esi(df:pd.DataFrame) -> pd.DataFrame:
    """
    Enriches the dataframe with concurrent ESI counts for each CaseID.
    Parameters:
        df (pd.DataFrame): The input dataframe containing 'CaseID', 'TIMESTAMP', 'TIMESTAMP_END', and 'ESI' columns.    
    Returns:
        pd.DataFrame: The enriched dataframe with additional columns for concurrent ESI counts.     
    """
    
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    df['TIMESTAMP_END'] = pd.to_datetime(df['TIMESTAMP_END'])
    df['TIMESTAMP_END'] = df['TIMESTAMP_END'].fillna(df['TIMESTAMP'])

    # Prepare columns for concurrent ESI counts
    for esi_val in range(1, 6):
        df[f'ESI_{esi_val}_concurrent'] = 0

    # Iterate through each row to count concurrent ESI values
    for idx, row in df.iterrows():
        case_id = row['CaseID']
        start = row['TIMESTAMP']
        end = row['TIMESTAMP_END']

        # Filter rows for the same CaseID
        same_case = df[df['CaseID'] == case_id]

        # Get overlapping rows
        overlapping = same_case[
            (same_case['TIMESTAMP'] <= end) & (same_case['TIMESTAMP_END'] >= start)
        ]

        # Cont count concurrent ESI values
        for esi_val in range(1, 6):
            count = (overlapping['ESI'] == esi_val).sum()
            df.at[idx, f'ESI_{esi_val}_concurrent'] = count

    return df

In [67]:
def enrich_day_columns(df:pd.DataFrame) -> pd.DataFrame:
    """
    Enriches the dataframe with day of the week columns in English (UK).
    Parameters:
        df (pd.DataFrame): The input dataframe containing 'TIMESTAMP' and 'TIMESTAMP_END' columns.
    Returns:
        pd.DataFrame: The enriched dataframe with 'DAY_start' and 'DAY_end' columns.
    """
    
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    df['TIMESTAMP_END'] = pd.to_datetime(df['TIMESTAMP_END'])

    df['DAY_start'] = df['TIMESTAMP'].dt.day_name(locale='en_GB')
    df['DAY_end'] = df['TIMESTAMP_END'].dt.day_name(locale='en_GB')
    return df

In [68]:
def enrich_shift_column(df:pd.DataFrame, shifts_dict:dict) -> pd.DataFrame:
    """
    Adds a SHIFT column to the dataframe based on TIMESTAMP and the provided shifts dictionary.
    Each shift is defined by a tuple (start_hour, end_hour) in 24h format.
    Parameters:
        df (pd.DataFrame): The input dataframe containing 'TIMESTAMP' column.
        shifts_dict (dict): A dictionary where keys are shift names and values are tuples of (start_hour, end_hour).
                            Example: {"Day": (7, 15), "Evening": (15, 22), "Night": (22, 7)}
    Returns:
        pd.DataFrame: The enriched dataframe with an additional 'SHIFT' column.
    """
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])

    def get_shift(dt):
        hour = dt.hour
        for shift_name, (start, end) in shifts_dict.items():
            if start < end:
                if start <= hour < end:
                    return shift_name
            else:  # Night shift (crosses midnight)
                if hour >= start or hour < end:
                    return shift_name
        return "Unknown"

    df['SHIFT'] = df['TIMESTAMP'].apply(get_shift)
    return df

In [69]:
def compute_path_complexity_v1(df: pd.DataFrame, case_col: str, activity_col: str, timestamp_col: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Compute graph-based complexity metrics per trace in an event log.

    Parameters:
    df (pd.DataFrame): Event log containing at least case ID, activity and timestamp columns.
    case_col (str): Name of the column identifying case IDs.
    activity_col (str): Name of the column identifying activity labels.
    timestamp_col (str): Name of the column with activity timestamps.

    Returns:
    tuple: (df_case_metrics, df_aggregated_stats)
           - df_case_metrics: DataFrame with complexity metrics per case
           - df_aggregated_stats: DataFrame with mean, std, min, max for each metric
    """
    # Ensure proper sorting
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    df = df.sort_values(by=[case_col, timestamp_col])

    results = []

    for case_id, group in df.groupby(case_col):
        activities = group[activity_col].tolist()
        unique_activities = set(activities)

        # Generate transitions between consecutive activities
        transitions = list(zip(activities[:-1], activities[1:]))
        unique_transitions = set(transitions)

        # Build directed graph of the trace
        num_nodes = len(unique_activities)
        num_edges = len(unique_transitions)
        max_edges = num_nodes * (num_nodes - 1) if num_nodes > 1 else 1
        density = num_edges / max_edges if max_edges > 0 else 0

        # Compute entropy over transitions
        transition_counts = Counter(transitions)
        total_transitions = sum(transition_counts.values())
        entropy = -sum((count / total_transitions) * log2(count / total_transitions)
                       for count in transition_counts.values())

        results.append({
            case_col: case_id,
            "num_activities": num_nodes,
            "num_transitions": num_edges,
            "density": density,
            "entropy": entropy
        })

    df_case_metrics = pd.DataFrame(results)
    df_aggregated_stats = df_case_metrics[["num_activities", "num_transitions", "density", "entropy"]].agg(['mean', 'std', 'min', 'max'])

    return df_case_metrics, df_aggregated_stats

In [70]:
def compute_path_complexity_v2(df: pd.DataFrame, case_col: str, activity_col: str, timestamp_col: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Compute graph-based complexity metrics per trace in an event log, including Cyclomatic Complexity.

    Parameters:
    df (pd.DataFrame): Event log containing at least case ID, activity and timestamp columns.
    case_col (str): Name of the column identifying case IDs.
    activity_col (str): Name of the column identifying activity labels.
    timestamp_col (str): Name of the column with activity timestamps.

    Returns:
    tuple: (df_case_metrics, df_aggregated_stats)
           - df_case_metrics: DataFrame with complexity metrics per case
           - df_aggregated_stats: DataFrame with mean, std, min, max for each metric
    """
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    df = df.sort_values(by=[case_col, timestamp_col])
    results = []

    for case_id, group in df.groupby(case_col):
        activities = group[activity_col].tolist()
        unique_activities = set(activities)

        # Build transitions and graph
        transitions = list(zip(activities[:-1], activities[1:]))
        unique_transitions = set(transitions)
        G = nx.DiGraph()
        G.add_edges_from(transitions)

        # Nodes and edges
        num_nodes = len(unique_activities)
        num_edges = len(unique_transitions)
        max_edges = num_nodes * (num_nodes - 1) if num_nodes > 1 else 1
        density = num_edges / max_edges if max_edges > 0 else 0

        # Entropy of transitions
        transition_counts = Counter(transitions)
        total_transitions = sum(transition_counts.values())
        entropy = -sum((count / total_transitions) * log2(count / total_transitions)
                       for count in transition_counts.values())

        # Cyclomatic Complexity
        P = nx.number_weakly_connected_components(G)  # Should be 1 in most cases
        cyclomatic = num_edges - num_nodes + 2 * P

        results.append({
            case_col: case_id,
            "num_activities": num_nodes,
            "num_transitions": num_edges,
            "density": density,
            "entropy": entropy,
            "cyclomatic_complexity": cyclomatic
        })

    df_case_metrics = pd.DataFrame(results)
    df_aggregated_stats = df_case_metrics[[
        "num_activities", "num_transitions", "density", "entropy", "cyclomatic_complexity"
    ]].agg(['mean', 'std', 'min', 'max']).round(2)

    return df_case_metrics, df_aggregated_stats

In [71]:
def plot_complexity_distributions(df_metrics: pd.DataFrame) -> None:
    """
    Plot histograms for complexity metrics computed per case.

    Parameters:
    df_metrics (pd.DataFrame): DataFrame containing per-case complexity metrics
                               with columns: 'num_activities', 'num_transitions', 
                               'density', and 'entropy'.
    """
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    axes = axes.flatten()
    columns = ["num_activities", "num_transitions", "density", "entropy"]

    for ax, col in zip(axes, columns):
        ax.hist(df_metrics[col], bins=20, color='skyblue', edgecolor='black')
        ax.set_title(f"Distribution of {col}")
        ax.set_xlabel(col)
        ax.set_ylabel("Frequency")

    plt.tight_layout()
    plt.show()

## MAIN

In [72]:
print("*** PROGRAM STARTED ***")
time_start = datetime.now()
print(f"Start time: {time_start.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Current working directory: {Path.cwd()}")
print(f"Data input directory: {DATA_IN_DIR}")
print(f"Data output directory: {DATA_OUT_DIR}")
print(f"CSV input file: {FILE_CSV_IN}")
print(f"XLS input file: {FILE_XLS_IN}")
print(f"CSV separator: {CSV_SEP}")
print(f"CSV key column: {LOG_KEY}")
print(f"XLS key column: {XLS_KEY}")
print(f"CSV data types file: {FILE_CSV_IN_DTYPE}")

*** PROGRAM STARTED ***
Start time: 2025-07-31 14:41:28
Current working directory: /Volumes/SAMSUNG-PHD/PhD/Articoli MIEI/PODS 2025/EVENT_LOG_ENRICHMENT
Data input directory: data_in
Data output directory: data_out
CSV input file: EVENT-LOG_ED_ENG.csv
XLS input file: Estrazione_Step_PS_ConSchedaCriptato.xls
CSV separator: ;
CSV key column: CaseID
XLS key column: Chiave
CSV data types file: EVENT-LOG_ED_ENG.json


### Outputs

In [73]:
create_output_directory(DATA_OUT_DIR)

> Creating directory...
Directory name: data_out
Directory already exists



### CSV data

In [74]:
print("> Reaging CSV data...")
print()
# Load the dictionary with the data types for each column
dtype_file_path = Path(DATA_IN_DIR) / FILE_CSV_IN_DTYPE
print(f"Loading CSV data types file: {dtype_file_path}")
df_dtype = load_json_to_dict(dtype_file_path)
print(f"Data types loaded: {df_dtype}")
print()
# Load the CSV file
csv_file_path = Path(DATA_IN_DIR) / FILE_CSV_IN
print(f"Loading CSV file: {csv_file_path}")
df_log_csv = pd.read_csv(csv_file_path, sep=CSV_SEP, dtype=df_dtype, encoding='utf-8', low_memory=False)

> Reaging CSV data...

Loading CSV data types file: data_in/EVENT-LOG_ED_ENG.json
Data types loaded: {'CaseID': 'object', 'ACTIVITY': 'object', 'TIMESTAMP': 'object', 'RESOURCE': 'object', 'ESI': 'int64', 'OUTCOME': 'object', 'INPAT-HOSP-DEP': 'object', 'CASE_DURATION_sec': 'float64', 'REMAINING_TIME_sec': 'float64', 'DATE': 'object'}

Loading CSV file: data_in/EVENT-LOG_ED_ENG.csv


In [75]:
# Show the shape of the dataframe
print(f"CSV file loaded with shape:{df_log_csv.shape}")

CSV file loaded with shape:(20624, 10)


In [76]:
# Show the first 5 rows of the dataframe
df_log_csv.head(5)

,CaseID,ACTIVITY,TIMESTAMP,RESOURCE,ESI,OUTCOME,INPAT-HOSP-DEP,CASE_DURATION_sec,REMAINING_TIME_sec,DATE
0,2022090001,TRIAGE,2022-09-01 00:03:55,NURS_0,3,A domicilio,-,41105.0,41105.0,2022-09-01
1,2022090001,PHYSICIAN-ASSESMENT,2022-09-01 00:22:00,DOCT_0,3,A domicilio,-,41105.0,40020.0,2022-09-01
2,2022090001,LABORATORY,2022-09-01 00:28:21,-,3,A domicilio,-,41105.0,39639.0,2022-09-01
3,2022090001,BRAIN CT SCAN,2022-09-01 00:38:00,-,3,A domicilio,-,41105.0,39060.0,2022-09-01
4,2022090001,CT SCAN,2022-09-01 00:38:00,-,3,A domicilio,-,41105.0,39060.0,2022-09-01


In [77]:
# Show the columns of the dataframe
print(f"Columns in the CSV file:\n{df_log_csv.columns.tolist()}")

Columns in the CSV file:
['CaseID', 'ACTIVITY', 'TIMESTAMP', 'RESOURCE', 'ESI', 'OUTCOME', 'INPAT-HOSP-DEP', 'CASE_DURATION_sec', 'REMAINING_TIME_sec', 'DATE']


In [78]:
# Distinct values in columns
list_col = [LOG_OUTCOME, LOG_ACTIVITY, "INPAT-HOSP-DEP"]
for i, col in enumerate(list_col):
    print(f"[{i}] - Distinct values in column '{col}'")
    print(f"{df_log_csv[col].unique().tolist()}")
    print()


[0] - Distinct values in column 'OUTCOME'
['A domicilio', 'Deceduto in P.S.', 'Ricoverato', 'Non risponde a chiamata', 'Chiede di essere dimessa contro il parere medico', 'Trasferito in altra struttura di ricovero', 'Si allontana senza consenso medico', 'Rientro in Reparto', 'Rifiuta ricovero', 'Trasferito in altra struttura RSA - RAF ecc', 'Inserito per errore']

[1] - Distinct values in column 'ACTIVITY'
['TRIAGE', 'PHYSICIAN-ASSESMENT', 'LABORATORY', 'BRAIN CT SCAN', 'CT SCAN', 'CONSULTATION', 'DISCHARGE', 'X-RAY', 'EKG', 'CHEST X-RAY / THORAX+LATERAL', 'RENAL ULTRASOUND', 'OBSERVATION UNIT', 'UPPER ABDOMINAL ULTRASOUND', 'EEG', 'PRE-TRANSFUSIONAL CHECKS', 'BED CHEST X-RAY', 'CAROTID DOPPLER ULTRASOUND', 'COMPLETE ABDOMINAL ULTRASOUND', 'MRI', 'SCROTAL/TESTICULAR ULTRASOUND', 'ESOPHAGOGASTRODUODENOSCOPY [EGD]', 'LOWER ABDOMINAL ULTRASOUND', 'LYMPH NODE BIOPSY', 'FIBEROPTIC BRONCHOSCOPY', 'CYTOLOGICAL SAMPLING', 'MICROBIOLOGICAL SAMPLING', 'BRONCHIAL BIOPSY/CYTOLOGY', 'VENOUS SAMPLIN

In [79]:
# Show the data types of the columns
print(f"Data types in the CSV file:\n{df_log_csv.dtypes}")

Data types in the CSV file:
CaseID                 object
ACTIVITY               object
TIMESTAMP              object
RESOURCE               object
ESI                     int64
OUTCOME                object
INPAT-HOSP-DEP         object
CASE_DURATION_sec     float64
REMAINING_TIME_sec    float64
DATE                   object
dtype: object


In [80]:
# Missing values
print(f"Missing values in the CSV file:\n{df_log_csv.isnull().sum()}")

Missing values in the CSV file:
CaseID                0
ACTIVITY              0
TIMESTAMP             0
RESOURCE              0
ESI                   0
OUTCOME               0
INPAT-HOSP-DEP        0
CASE_DURATION_sec     0
REMAINING_TIME_sec    0
DATE                  0
dtype: int64


In [81]:
print("Checking case length...")
df_log_csv_case_length = count_rows_per_case(df_log_csv, LOG_KEY, LOG_KEY) # count rows per case and rename the output column to CSV_KEY
df_log_csv_case_length['CaseID'] = df_log_csv_case_length['CaseID'].astype(object)
ncases_csv = df_log_csv_case_length[LOG_KEY].nunique()
ncases_csv_list = df_log_csv_case_length[LOG_KEY].unique().tolist()
print(f"Number of unique cases in the CSV file: {ncases_csv}")

Checking case length...
Number of unique cases in the CSV file: 3478


In [82]:
df_log_csv_case_length.head(5)

,CaseID,case_row_count
0,2022090001,11
1,2022090002,4
2,2022090003,5
3,2022090004,5
4,2022090005,5


In [83]:
print(f"Mean {round(df_log_csv_case_length["case_row_count"].mean(),2)}")
print(f"Median {df_log_csv_case_length["case_row_count"].median()}")
print(f"Minimum {df_log_csv_case_length["case_row_count"].min()}")
print(f"Maximum {df_log_csv_case_length["case_row_count"].max()}")
print(f"Standard deviation {round(df_log_csv_case_length['case_row_count'].std(), 2)}")

Mean 5.93
Median 5.0
Minimum 3
Maximum 32
Standard deviation 3.43


In [84]:
# Save the case length dataframe to a CSV file
case_length_file_path = Path(DATA_OUT_DIR) / f"{Path(FILE_CSV_IN).stem}_stats_case_length.csv"
df_log_csv_case_length.to_csv(case_length_file_path, sep=CSV_SEP, index=False)
print(f"Case length saved to: {case_length_file_path}")

Case length saved to: data_out/EVENT-LOG_ED_ENG_stats_case_length.csv


### XLS data

In [85]:
print("> Reading XLS data...")
print()
# Load the dictionary with the data types for each column
dtype_file_path = Path(DATA_IN_DIR) / FILE_XLS_IN_DTYPE
print(f"Loading XLS data types file: {dtype_file_path}")
df_dtype = load_json_to_dict(dtype_file_path)
print(f"Data types loaded: {df_dtype}")
print()
# Load the XLS file
xls_file_path = Path(DATA_IN_DIR) / FILE_XLS_IN
print(f"Loading XLS file: {xls_file_path}")
# df_log_xls = pd.read_excel(xls_file_path, dtype=df_dtype, engine='openpyxl') # for XLSX files
df_log_xls = pd.read_excel(xls_file_path, dtype=df_dtype, engine='xlrd') # for XLS files

> Reading XLS data...

Loading XLS data types file: data_in/Estrazione_Step_PS_ConSchedaCriptato.json
Data types loaded: {'Chiave': 'object', 'Evento': 'object', 'Priorità al Triage in Ingresso': 'object', 'Medico accettante': 'object', 'Infermiere di triage': 'object', 'Prestazioni': 'object', 'Data inizio': 'datetime64[ns]', 'Data Fine_Referto': 'datetime64[ns]', 'Esito dimissione': 'object', 'Ricoverabile dal': 'datetime64[ns]', 'Reparto di ricovero': 'object'}

Loading XLS file: data_in/Estrazione_Step_PS_ConSchedaCriptato.xls


In [86]:
# Show the shape of the dataframe
print(f"XLS file loaded with shape:{df_log_xls.shape}")

XLS file loaded with shape:(20798, 9)


In [87]:
# Show the first 5 rows of the dataframe
df_log_xls.head(5)

,Chiave,Evento,Priorità al Triage in Ingresso,Prestazioni,Data inizio,Data Fine_Referto,Esito dimissione,Ricoverabile dal,Reparto di ricovero
0,2022090001,1 - Triage,3-Azzurro,NaN,2022-09-01 00:03:55,2022-09-01 00:22:00,NaN,NaT,NaN
1,2022090001,2 - Presa in carico,NaN,NaN,2022-09-01 00:22:00,2022-09-01 11:29:00,NaN,NaT,NaN
2,2022090001,3 - Laboratorio,NaN,NaN,2022-09-01 00:28:21,NaT,NaN,NaT,NaN
3,2022090001,3 - Laboratorio,NaN,NaN,2022-09-01 07:00:00,NaT,NaN,NaT,NaN
4,2022090001,4 - Consulenza,NaN,NaN,2022-09-01 07:22:30,2022-09-01 09:11:20,NaN,NaT,NaN


In [88]:
# Show the columns of the dataframe
print(f"Columns in the XLS file:\n{df_log_xls.columns.tolist()}")

Columns in the XLS file:
['Chiave', 'Evento', 'Priorità al Triage in Ingresso', 'Prestazioni', 'Data inizio', 'Data Fine_Referto', 'Esito dimissione', 'Ricoverabile dal', 'Reparto di ricovero']


In [89]:
# Show the data types of the columns
print(f"Data types in the XLS file:\n{df_log_xls.dtypes}")

Data types in the XLS file:
Chiave                                    object
Evento                                    object
Priorità al Triage in Ingresso            object
Prestazioni                               object
Data inizio                       datetime64[ns]
Data Fine_Referto                 datetime64[ns]
Esito dimissione                          object
Ricoverabile dal                  datetime64[ns]
Reparto di ricovero                       object
dtype: object


In [90]:
# Missing values
print(f"Missing values in the XLS file:\n{df_log_xls.isnull().sum()}")

Missing values in the XLS file:
Chiave                                0
Evento                                0
Priorità al Triage in Ingresso    17319
Prestazioni                       14450
Data inizio                           0
Data Fine_Referto                  6435
Esito dimissione                  17319
Ricoverabile dal                  20276
Reparto di ricovero               20342
dtype: int64


In [91]:
print("Checking case length...")
df_log_xls_case_length = count_rows_per_case(df_log_xls, XLS_KEY, LOG_KEY) # count rows per case and rename the output column to CSV_KEY
df_log_xls_case_length['CaseID'] = df_log_xls_case_length['CaseID'].astype(object)
ncases_xls = df_log_xls_case_length[LOG_KEY].nunique()
ncases_xls_list = df_log_xls_case_length[LOG_KEY].unique().tolist()
print(f"Number of unique cases in the XLS file: {ncases_xls}")

Checking case length...
Number of unique cases in the XLS file: 3479


In [92]:
# Save the case length dataframe to a CSV file
case_length_file_path = Path(DATA_OUT_DIR) / f"{Path(FILE_XLS_IN).stem}_case_length.csv"
df_log_xls_case_length.to_csv(case_length_file_path, sep=CSV_SEP, index=False)
print(f"Case length saved to: {case_length_file_path}")

Case length saved to: data_out/Estrazione_Step_PS_ConSchedaCriptato_case_length.csv


In [93]:
# Check for differences in case IDs between CSV and XLS
print("> Checking differences between CSV and XLS data...")
if ncases_xls == ncases_csv:
    print("The values are equal.")
elif ncases_xls > ncases_csv:
    print("ncases_xls is greater than ncases_csv.")
else:
    print("ncases_csv is greater than ncases_xls.")

> Checking differences between CSV and XLS data...
ncases_xls is greater than ncases_csv.


In [94]:
case_diff = list(set(ncases_csv_list).symmetric_difference(set(ncases_xls_list)))
print(f"Cases differences {len(case_diff)}: {case_diff}")
path_out = Path(DATA_OUT_DIR) / f"{Path(FILE_XLS_IN).stem}_vs_{Path(FILE_CSV_IN).stem}_case_missing.csv"
df_case_diff = pd.DataFrame({'CaseID': case_diff})
df_case_diff.to_csv(path_out, index=False)
print(f"Case differences saved to: {path_out}")

Cases differences 1: ['2022090500']
Case differences saved to: data_out/Estrazione_Step_PS_ConSchedaCriptato_vs_EVENT-LOG_ED_ENG_case_missing.csv


In [95]:
# Differences in case IDs between CSV and XLS

# Rename the 'row_count' columns before the join
df_log_xls_case_length = df_log_xls_case_length.rename(columns={'case_row_count': 'row_count_xls'})
df_log_csv_case_length = df_log_csv_case_length.rename(columns={'case_row_count': 'row_count_csv'})

# Perform the join on the 'CaseID' column
df_merged_diff = pd.merge(df_log_xls_case_length, df_log_csv_case_length, on='CaseID', how='inner')

# Calculate the difference between the two row counts
df_merged_diff['row_count_diff'] = df_merged_diff['row_count_xls'] - df_merged_diff['row_count_csv']

# Save the result to a CSV file
path_out = Path(DATA_OUT_DIR) / f"{Path(FILE_XLS_IN).stem}_vs_{Path(FILE_CSV_IN).stem}_case_length_diff.csv"
print(f"Saving the differences to: {path_out}")
df_merged_diff.to_csv(path_out, sep=CSV_SEP, index=False)

Saving the differences to: data_out/Estrazione_Step_PS_ConSchedaCriptato_vs_EVENT-LOG_ED_ENG_case_length_diff.csv


In [96]:
# Filter the cases where row_count_diff > 0
df_diff_positive = df_merged_diff[df_merged_diff['row_count_diff'] > 0]

# Extract the CaseID values into a list
case_ids_with_positive_diff = df_diff_positive['CaseID'].tolist()

# Display the number of cases and the list
print(f"Number of CaseIDs with row_count_diff > 0: {len(case_ids_with_positive_diff)}")

Number of CaseIDs with row_count_diff > 0: 122


### ADD TIMESTAMP_END

In [97]:
# Step 1: insert the "TIMESTAMP_END" column immediately after the "TIMESTAMP" column
if 'TIMESTAMP' in df_log_csv.columns:
    timestamp_index = df_log_csv.columns.get_loc('TIMESTAMP')  # Get the index of the "TIMESTAMP" column
    df_log_csv.insert(timestamp_index + 1, 'TIMESTAMP_END', None)  # Insert "TIMESTAMP_END" after "TIMESTAMP"
else:
    print("The column 'TIMESTAMP' does not exist in df_log_csv.")

In [98]:
# Step 2: Create a dictionary from df_log_xls to map case_id -> End Report Date
col_end_date = 'Data Fine_Referto'
case_id_to_end_date = df_log_xls.groupby(XLS_KEY)[col_end_date].apply(list).to_dict()

In [99]:
i = 0
skipped_case_ids = []
for case_id, group in df_log_csv.groupby(LOG_KEY):
    i += 1
    print(f"Processing [{i}/{len(df_log_csv_case_length)}]: {case_id}")
    if case_id in case_id_to_end_date:
        end_dates = case_id_to_end_date[case_id]
        if len(group) == len(end_dates):
            # If the number of rows matches, assign the values directly
            df_log_csv.loc[group.index, 'TIMESTAMP_END'] = end_dates
        else:
            # Skip case_ids with mismatched row counts
            print(f"Skipping case_id {case_id} due to mismatched row counts.")
            skipped_case_ids.append(case_id)

Processing [1/3478]: 2022090001
Processing [2/3478]: 2022090002
Processing [3/3478]: 2022090003
Processing [4/3478]: 2022090004
Processing [5/3478]: 2022090005
Processing [6/3478]: 2022090006
Processing [7/3478]: 2022090007
Processing [8/3478]: 2022090008
Processing [9/3478]: 2022090009
Processing [10/3478]: 2022090010
Processing [11/3478]: 2022090011
Processing [12/3478]: 2022090012
Processing [13/3478]: 2022090013
Processing [14/3478]: 2022090014
Processing [15/3478]: 2022090015
Processing [16/3478]: 2022090016
Processing [17/3478]: 2022090017
Processing [18/3478]: 2022090018
Processing [19/3478]: 2022090019
Processing [20/3478]: 2022090020
Processing [21/3478]: 2022090021
Processing [22/3478]: 2022090022
Processing [23/3478]: 2022090023
Processing [24/3478]: 2022090024
Processing [25/3478]: 2022090025
Processing [26/3478]: 2022090026
Processing [27/3478]: 2022090027
Processing [28/3478]: 2022090028
Processing [29/3478]: 2022090029
Processing [30/3478]: 2022090030
Processing [31/3478

In [100]:
print(f"Skipped case_ids ({len(skipped_case_ids)}): {skipped_case_ids}")
ratio_skipped = len(skipped_case_ids) / len(df_log_csv_case_length)
print(f"Ratio of skipped case_ids: {ratio_skipped:.2%}")

Skipped case_ids (122): ['2022090078', '2022090201', '2022090227', '2022090243', '2022090252', '2022090257', '2022090283', '2022090290', '2022090327', '2022090346', '2022090368', '2022090370', '2022090382', '2022090414', '2022090416', '2022090468', '2022090484', '2022090529', '2022090532', '2022090600', '2022090614', '2022090666', '2022090685', '2022090688', '2022090699', '2022090791', '2022090803', '2022090805', '2022090823', '2022090842', '2022090895', '2022090973', '2022091030', '2022091043', '2022091081', '2022091096', '2022091104', '2022091118', '2022091240', '2022091322', '2022091351', '2022091356', '2022091363', '2022091365', '2022091443', '2022091458', '2022091475', '2022091485', '2022091487', '2022091556', '2022091570', '2022091661', '2022091670', '2022091716', '2022091718', '2022091721', '2022091724', '2022091728', '2022091734', '2022091737', '2022091750', '2022091755', '2022091757', '2022091759', '2022091764', '2022091770', '2022091774', '2022091779', '2022091784', '20220917

In [101]:
# Save skipped case_ids to a CSV file
skipped_case_ids_file_path = Path(DATA_OUT_DIR) / "timestamp_end_skipped_case_ids.csv"
pd.DataFrame({'CaseID': skipped_case_ids}).to_csv(skipped_case_ids_file_path, index=True, index_label="Num")
print(f"Skipped case_ids saved to: {skipped_case_ids_file_path}")

Skipped case_ids saved to: data_out/timestamp_end_skipped_case_ids.csv


In [102]:
df_log_csv.head()

,CaseID,ACTIVITY,TIMESTAMP,TIMESTAMP_END,RESOURCE,ESI,OUTCOME,INPAT-HOSP-DEP,CASE_DURATION_sec,REMAINING_TIME_sec,DATE
0,2022090001,TRIAGE,2022-09-01 00:03:55,2022-09-01 00:22:00,NURS_0,3,A domicilio,-,41105.0,41105.0,2022-09-01
1,2022090001,PHYSICIAN-ASSESMENT,2022-09-01 00:22:00,2022-09-01 11:29:00,DOCT_0,3,A domicilio,-,41105.0,40020.0,2022-09-01
2,2022090001,LABORATORY,2022-09-01 00:28:21,NaT,-,3,A domicilio,-,41105.0,39639.0,2022-09-01
3,2022090001,BRAIN CT SCAN,2022-09-01 00:38:00,NaT,-,3,A domicilio,-,41105.0,39060.0,2022-09-01
4,2022090001,CT SCAN,2022-09-01 00:38:00,2022-09-01 09:11:20,-,3,A domicilio,-,41105.0,39060.0,2022-09-01


In [103]:
df_log_csv.shape

(20624, 11)

In [104]:
df_log_csv["ESI"].unique().tolist()

[3, 4, 5, 1, 2]

In [105]:
# Save the CSV file
# enriched_csv_file_path = Path(DATA_OUT_DIR) / f"{Path(FILE_CSV_IN).stem}_enriched.csv"
csv_file_path = Path(DATA_OUT_DIR) / f"_{Path(FILE_CSV_IN).stem}_base.csv"
df_log_csv.to_csv(csv_file_path, sep=CSV_SEP, index=False)
print(f"Event log CSV file saved to: {csv_file_path}")

Event log CSV file saved to: data_out/_EVENT-LOG_ED_ENG_base.csv


### OUTCOME stats

In [106]:
print("> Outcome stats")
print("Distinct cases in the CSV file:", df_log_csv[LOG_KEY].nunique())
df_log_csv_outcome = df_log_csv.drop_duplicates(subset=[LOG_KEY])
outcome_stats = (
    df_log_csv_outcome[LOG_OUTCOME]
    .value_counts(dropna=False)
    .reset_index()
    .rename(columns={"index": "outcome", LOG_OUTCOME: "count"})
)
outcome_stats["percent"] = (outcome_stats["count"] / outcome_stats["count"].sum() * 100).round(2)
outcome_stats

> Outcome stats
Distinct cases in the CSV file: 3478


,outcome,count,percent
0,A domicilio,2807,80.71
1,Ricoverato,456,13.11
2,Non risponde a chiamata,106,3.05
3,Chiede di essere dimessa contro il parere medico,34,0.98
4,Si allontana senza consenso medico,24,0.69
5,Trasferito in altra struttura di ricovero,22,0.63
6,Deceduto in P.S.,12,0.35
7,Rientro in Reparto,9,0.26
8,Rifiuta ricovero,6,0.17
9,Trasferito in altra struttura RSA - RAF ecc,1,0.03


In [107]:
path_stats = Path(DATA_OUT_DIR) / f"{Path(FILE_CSV_IN).stem}_stats_outcome.csv"
outcome_stats.to_csv(path_stats, sep=CSV_SEP, index=False)
print(f"Outcome stats saved to: {path_stats}")

Outcome stats saved to: data_out/EVENT-LOG_ED_ENG_stats_outcome.csv


### COMPLEXITY

In [108]:
print("> Complexity stats")
# Compute complexity
df_case_metrics, df_case_metrics_aggregated = compute_path_complexity_v2(df_log_csv, case_col=LOG_KEY, activity_col=LOG_ACTIVITY, timestamp_col=LOG_TIMESTAMP)

# Display aggregated statistics
print(df_case_metrics_aggregated)

> Complexity stats
      num_activities  num_transitions  density  entropy  cyclomatic_complexity
mean            4.86             4.51     0.25     1.92                   1.65
std             1.78             2.78     0.06     0.74                   1.22
min             3.00             2.00     0.07     0.99                   1.00
max            18.00            21.00     0.35     4.39                  11.00


In [121]:
# Join outcome to per-case metrics
df_outcome = df_log_csv[[LOG_KEY, LOG_OUTCOME]].drop_duplicates()
df_case_metrics_outcome = df_case_metrics.merge(df_outcome, left_on=LOG_KEY, right_on=LOG_KEY, how="left").drop(columns=LOG_KEY)

# Compute aggregated statistics grouped by outcome
grouped_stats = df_case_metrics_outcome.groupby(LOG_OUTCOME)[["num_activities", "num_transitions", "density", "entropy", "cyclomatic_complexity"]].agg(['mean', 'std', 'min', 'max']).round(2)

grouped_stats

num_activities                \
                                                           mean   std min max   
OUTCOME                                                                         
A domicilio                                                4.55  1.50   3  14   
Chiede di essere dimessa contro il parere medico           5.71  1.73   3   9   
Deceduto in P.S.                                           5.92  2.35   3  11   
Inserito per errore                                        3.00   NaN   3   3   
Non risponde a chiamata                                    3.10  0.31   3   4   
Ricoverato                                                 7.06  1.78   3  18   
Rientro in Reparto                                         3.56  0.73   3   5   
Rifiuta ricovero                                           6.00  1.10   5   8   
Si allontana senza consenso medico                         3.46  0.59   3   5   
Trasferito in altra struttura RSA - RAF ecc                7.00   NaN   7   7   
Trasferito in altra struttura di ricovero                  6.86  2.36   3  12   

                                                 num_transitions            \
                                                            mean   std min   
OUTCOME                                                                      
A domicilio                                                 3.99  2.19   2   
Chiede di essere dimessa contro il parere medico            6.00  3.04   2   
Deceduto in P.S.                                            6.50  4.40   2   
Inserito per errore                                         2.00   NaN   2   
Non risponde a chiamata                                     2.12  0.38   2   
Ricoverato                                                  8.08  3.25   2   
Rientro in Reparto                                          2.67  0.87   2   
Rifiuta ricovero                                            5.83  1.83   4   
Si allontana senza consenso medico                          2.54  0.88   2   
Trasferito in altra struttura RSA - RAF ecc                 9.00   NaN   9   
Trasferito in altra struttura di ricovero                   7.73  4.12   2   

                                                     density              \
                                                 max    mean   std   min   
OUTCOME                                                                    
A domicilio                                       19    0.26  0.06  0.08   
Chiede di essere dimessa contro il parere medico  13    0.23  0.05  0.12   
Deceduto in P.S.                                  15    0.23  0.07  0.14   
Inserito per errore                                2    0.33   NaN  0.33   
Non risponde a chiamata                            4    0.33  0.02  0.25   
Ricoverato                                        21    0.19  0.05  0.07   
Rientro in Reparto                                 4    0.30  0.05  0.20   
Rifiuta ricovero                                   9    0.20  0.04  0.16   
Si allontana senza consenso medico                 6    0.30  0.04  0.25   
Trasferito in altra struttura RSA - RAF ecc        9    0.21   NaN  0.21   
Trasferito in altra struttura di ricovero         17    0.20  0.06  0.08   

                                                       entropy              \
                                                   max    mean   std   min   
OUTCOME                                                                      
A domicilio                                       0.35    1.80  0.64  0.99   
Chiede di essere dimessa contro il parere medico  0.33    2.35  0.74  1.00   
Deceduto in P.S.                                  0.33    2.39  0.91  1.00   
Inserito per errore                               0.33    1.00   NaN  1.00   
Non risponde a chiamata                           0.33    1.06  0.19  1.00   
Ricoverato                                        0.35    2.85  0.58  1.00   
Rientro in Reparto                                0.33

In [110]:
# Save the aggregated statistics to a CSV file
print("> Saving aggregated statistics by outcome...")
path_outcome_stats = Path(DATA_OUT_DIR) / f"{Path(FILE_CSV_IN).stem}_stats_complexity_by_outcome.csv"
grouped_stats.to_csv(path_outcome_stats, sep=CSV_SEP)
print(f"Outcome stats saved to: {path_outcome_stats}")

> Saving aggregated statistics by outcome...
Outcome stats saved to: data_out/EVENT-LOG_ED_ENG_stats_complexity_by_outcome.csv


### LOG ENRICHMENT

In [111]:
print("> Enriching data")
df_log_csv_enriched = df_log_csv.copy()  # Create a copy of the enriched dataframe

> Enriching data


#### DAY

In [112]:
print("> Enriching data with DAYS...")
df_log_csv_enriched = enrich_day_columns(df_log_csv_enriched)

> Enriching data with DAYS...


In [113]:
df_log_csv_enriched.head()

,CaseID,ACTIVITY,TIMESTAMP,TIMESTAMP_END,RESOURCE,ESI,OUTCOME,INPAT-HOSP-DEP,CASE_DURATION_sec,REMAINING_TIME_sec,DATE,DAY_start,DAY_end
0,2022090001,TRIAGE,2022-09-01 00:03:55,2022-09-01 00:22:00,NURS_0,3,A domicilio,-,41105.0,41105.0,2022-09-01,Thursday,Thursday
1,2022090001,PHYSICIAN-ASSESMENT,2022-09-01 00:22:00,2022-09-01 11:29:00,DOCT_0,3,A domicilio,-,41105.0,40020.0,2022-09-01,Thursday,Thursday
2,2022090001,LABORATORY,2022-09-01 00:28:21,NaT,-,3,A domicilio,-,41105.0,39639.0,2022-09-01,Thursday,NaN
3,2022090001,BRAIN CT SCAN,2022-09-01 00:38:00,NaT,-,3,A domicilio,-,41105.0,39060.0,2022-09-01,Thursday,NaN
4,2022090001,CT SCAN,2022-09-01 00:38:00,2022-09-01 09:11:20,-,3,A domicilio,-,41105.0,39060.0,2022-09-01,Thursday,Thursday


## Concurrent ESI

In [114]:
print("> Enriching data with concurrent ESI...")
df_log_csv_enriched = enrich_concurrent_esi(df_log_csv_enriched)

> Enriching data with concurrent ESI...


In [115]:
df_log_csv_enriched.head()

,CaseID,ACTIVITY,TIMESTAMP,TIMESTAMP_END,RESOURCE,ESI,OUTCOME,INPAT-HOSP-DEP,CASE_DURATION_sec,REMAINING_TIME_sec,DATE,DAY_start,DAY_end,ESI_1_concurrent,ESI_2_concurrent,ESI_3_concurrent,ESI_4_concurrent,ESI_5_concurrent
0,2022090001,TRIAGE,2022-09-01 00:03:55,2022-09-01 00:22:00,NURS_0,3,A domicilio,-,41105.0,41105.0,2022-09-01,Thursday,Thursday,0,0,2,0,0
1,2022090001,PHYSICIAN-ASSESMENT,2022-09-01 00:22:00,2022-09-01 11:29:00,DOCT_0,3,A domicilio,-,41105.0,40020.0,2022-09-01,Thursday,Thursday,0,0,11,0,0
2,2022090001,LABORATORY,2022-09-01 00:28:21,2022-09-01 00:28:21,-,3,A domicilio,-,41105.0,39639.0,2022-09-01,Thursday,NaN,0,0,2,0,0
3,2022090001,BRAIN CT SCAN,2022-09-01 00:38:00,2022-09-01 00:38:00,-,3,A domicilio,-,41105.0,39060.0,2022-09-01,Thursday,NaN,0,0,6,0,0
4,2022090001,CT SCAN,2022-09-01 00:38:00,2022-09-01 09:11:20,-,3,A domicilio,-,41105.0,39060.0,2022-09-01,Thursday,Thursday,0,0,8,0,0


## SHIFT

In [116]:
print("> Enriching data with SHIFTS...")
df_log_csv_enriched = enrich_shift_column(df_log_csv_enriched, shifts_dict)

> Enriching data with SHIFTS...


In [117]:
df_log_csv_enriched.head()

,CaseID,ACTIVITY,TIMESTAMP,TIMESTAMP_END,RESOURCE,ESI,OUTCOME,INPAT-HOSP-DEP,CASE_DURATION_sec,REMAINING_TIME_sec,DATE,DAY_start,DAY_end,ESI_1_concurrent,ESI_2_concurrent,ESI_3_concurrent,ESI_4_concurrent,ESI_5_concurrent,SHIFT
0,2022090001,TRIAGE,2022-09-01 00:03:55,2022-09-01 00:22:00,NURS_0,3,A domicilio,-,41105.0,41105.0,2022-09-01,Thursday,Thursday,0,0,2,0,0,Night
1,2022090001,PHYSICIAN-ASSESMENT,2022-09-01 00:22:00,2022-09-01 11:29:00,DOCT_0,3,A domicilio,-,41105.0,40020.0,2022-09-01,Thursday,Thursday,0,0,11,0,0,Night
2,2022090001,LABORATORY,2022-09-01 00:28:21,2022-09-01 00:28:21,-,3,A domicilio,-,41105.0,39639.0,2022-09-01,Thursday,NaN,0,0,2,0,0,Night
3,2022090001,BRAIN CT SCAN,2022-09-01 00:38:00,2022-09-01 00:38:00,-,3,A domicilio,-,41105.0,39060.0,2022-09-01,Thursday,NaN,0,0,6,0,0,Night
4,2022090001,CT SCAN,2022-09-01 00:38:00,2022-09-01 09:11:20,-,3,A domicilio,-,41105.0,39060.0,2022-09-01,Thursday,Thursday,0,0,8,0,0,Night


In [118]:
# Save the CSV file
enriched_csv_file_path = Path(DATA_OUT_DIR) / f"_{Path(FILE_CSV_IN).stem}_enriched.csv"
df_log_csv_enriched.to_csv(enriched_csv_file_path, sep=CSV_SEP, index=False)
print(f"Event log CSV file saved to: {enriched_csv_file_path}")

Event log CSV file saved to: data_out/_EVENT-LOG_ED_ENG_enriched.csv


In [119]:
# Complexity analysis
print("> Complexity analysis")

> Complexity analysis


In [120]:
print("*** PROGRAM ENDED ***")
time_end = datetime.now()  
delta_time = time_end - time_start
print(f"End time: {time_end.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Elapsed time: {delta_time}")
print(f"Elapsed time in seconds: {delta_time.total_seconds()}")
print(f"Elapsed time in minutes: {delta_time.total_seconds() / 60}")
print(f"Elapsed time in hours: {delta_time.total_seconds() / 3600}")
print("*" * 6)

*** PROGRAM ENDED ***
End time: 2025-07-31 14:41:59
Elapsed time: 0:00:30.994830
Elapsed time in seconds: 30.99483
Elapsed time in minutes: 0.5165805
Elapsed time in hours: 0.008609675
******
